In [1]:
import sys
sys.path.append("../Evaluation")

In [3]:
ls ../Evaluation/evaluation

__init__.py  __pycache__/  test.py


In [7]:
from evaluation.test import test

In [9]:
test(1).DGPSampling.all()

<class 'maccabee.constants.Constants'>


AttributeError: 'NoneType' object has no attribute 'DGPSampling'